In [1]:
import requests
import bs4
import json
import pandas as pd
import time

In [2]:
def fetch(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    return requests.get(url, headers=headers, timeout=10)

In [3]:
results = []
for i in range(1,62,1):
    print(f"Grammy - {i}")
    try:
        if i % 10 == 1 and i != 11:
            url = 'https://www.grammy.com/grammys/awards/{}st-annual-grammy-awards'.format(i)
        elif i % 10 == 2 and i != 12:
            url = 'https://www.grammy.com/grammys/awards/{}nd-annual-grammy-awards'.format(i)
        elif i % 10 == 3 and i != 13:
            url = 'https://www.grammy.com/grammys/awards/{}rd-annual-grammy-awards'.format(i)
        else:
            url = 'https://www.grammy.com/grammys/awards/{}th-annual-grammy-awards'.format(i)
        res = fetch(url)
        page = bs4.BeautifulSoup(res.text, 'lxml')
        for award in page.select(".view-grouping"):
            try:
                category = award.select(".view-grouping-header")[0].getText()
                info = {"annualGrammy": i, "category": category}
                try:
                    winner = award.select("p")[0].select(".freelink-internal")[0].getText()
                    individualAward = False
                except:
                    winner = award.select(".views-field-title")[0].getText()
                    individualAward = True
                    info['awardType'] = 'Individual'
                    info['awardFor'] = winner

                if individualAward == False:
                    info['awardType'] = 'Work'
                    workTitle = award.select(".views-field-title")[0].getText()
                    info['awardFor'] = workTitle
                try:
                    name = award.select(".views-field-field-description")[0].getText()
                    info['name'] = name.strip()
                except:
                    info['name'] = winner
                if info['name'] == info['awardFor']:
                    continue
                results.append(info)
            except:
                print("Error with award on {}".format(i))
        time.sleep(1)
    except:
        import traceback
        print("Problem with {}".format(i))
        traceback.print_tb()
df = pd.DataFrame(results)
df.head()

Grammy - 1
Grammy - 2
Grammy - 3
Grammy - 4
Grammy - 5
Grammy - 6
Grammy - 7
Grammy - 8
Grammy - 9
Grammy - 10
Grammy - 11
Grammy - 12
Grammy - 13
Grammy - 14
Grammy - 15
Grammy - 16
Grammy - 17
Grammy - 18
Grammy - 19
Grammy - 20
Grammy - 21
Grammy - 22
Grammy - 23
Grammy - 24
Grammy - 25
Grammy - 26
Grammy - 27
Grammy - 28
Grammy - 29
Grammy - 30
Grammy - 31
Grammy - 32
Grammy - 33
Grammy - 34
Grammy - 35
Grammy - 36
Grammy - 37
Grammy - 38
Grammy - 39
Grammy - 40
Grammy - 41
Grammy - 42
Grammy - 43
Grammy - 44
Grammy - 45
Grammy - 46
Grammy - 47
Grammy - 48
Grammy - 49
Grammy - 50
Grammy - 51
Grammy - 52
Grammy - 53
Grammy - 54
Grammy - 55
Grammy - 56
Grammy - 57
Grammy - 58
Grammy - 59
Grammy - 60
Grammy - 61


,annualGrammy,awardFor,awardType,category,name
0,1,Nel Blu Dipinto Di Blu (Volare),Individual,Record Of The Year,Domenico Modugno
1,1,The Music From Peter Gunn,Work,Album Of The Year,Henry Mancini
2,1,Nel Blu Dipinto Di Blu (Volare),Individual,Song Of The Year,"Domenico Modugno, songwriter (Domenico Modugno)"
3,1,Ella Fitzgerald Sings The Irving Berlin Song Book,Work,"Best Vocal Performance, Female",Ella Fitzgerald
4,1,Catch A Falling Star,Individual,"Best Vocal Performance, Male",Perry Como


In [4]:
df.to_csv("../data/grammy-winners.csv", index=False)